# The instability of numerical codes: symptoms

The instability of numerical schemes and codes is one of the critical problems that a researcher using numerical experimentation may encounter. In this exercise, we are going to see how violent numerical instabilities can be.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from importlib import reload
import import_ipynb

from ex_2b import get_uu_t0, get_u_exact
from nm_lib import nm_lib as nm
# Define some simple global matplotlib settings
plt.rcParams.update(
    {"font.size" : "15",
     "font.family"          : "STIXGeneral",
     "mathtext.fontset"     : "stix",
     "figure.autolayout"    : "True",
     "figure.figsize"       : (7,7),
     "lines.markersize"     : 10,
     }
)

def animate_u(uunt: np.ndarray, xx: np.ndarray, tt: np.ndarray, a: float, exact: callable = None, initial = None) -> None:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
    title_str_app = f" a={a:.2f}" if a is not None else ""
    print(f"Animation with nint={len(xx)-1:d}"+title_str_app+f", nframes={len(tt):d}:")

    def init(): 
        ax.plot(xx,uunt[0,:])

    def animate(i):
        ax.clear()
        ax.plot(xx,uunt[i,:], label="uunt")
        if initial is None:
            ax.plot(xx,uunt[0,:], ls=":", label="init")
        else:
            ax.plot(xx, initial, ls=":", label="init")
        if exact is not None:
            ax.plot(xx, exact(xx, tt[i],a),ls="--", label="exact")
        ax.set_title(f"t={tt[i]:.2f}, nint={len(xx)-1:d}"+title_str_app)
        ax.legend(loc=1)

    anim = FuncAnimation(fig, animate, interval=200, frames=len(tt), init_func=init)
    html = HTML(anim.to_jshtml())
    display(html)
    plt.close()

def analyse_instability(nint: int, nt: int = 60, x0: float = -2.6, xf: float = 2.6, a=1, crit_value: float = 1.5, **kwargs):
    xx, dx = nm.get_xx(nint, x0, xf)

    # Handle additional keyword arguments
    ddx = kwargs["ddx"] if "ddx" in kwargs else nm.deriv_dnw
    bnd_limits = kwargs["bnd_limits"] if "bnd_limits" in kwargs else [0,1]
    cfl_cut = kwargs["cfl_cut"] if "cfl_cut" in kwargs else 0.98
    ani = kwargs["animate"] if "animate" in kwargs else True
    evolv_func = kwargs["evolv_func"] if "evolv_func" in kwargs else nm.evolv_adv_burgers
    init_func = kwargs["init_func"] if "init_func" in kwargs else get_uu_t0
    u_exact = kwargs["u_exact"] if "u_exact" in kwargs else get_u_exact

    uut0 = init_func(xx)
    if a is None:
        tt, uunt = evolv_func(xx, uut0, nt, ddx=ddx, bnd_limits=bnd_limits, cfl_cut=cfl_cut)
    else:
        tt, uunt = evolv_func(xx, uut0, nt, a, ddx=ddx, bnd_limits=bnd_limits, cfl_cut=cfl_cut)
    # check for instability:
    crit_value = crit_value    # I say instability occurs if any values in u(x,t) is larger than this value
    id_crit = np.argmax(np.max(np.abs(uunt),axis=1)>crit_value) # find first idx of time where the max abs value in uunt is larger than crit_value
    print(f"Using {ddx.__name__}, with {nint:d} intervals, cfl_cut={cfl_cut:.2f}, nt={nt:d}")
    if id_crit == 0:
        print(f"Instability not found for t<{tt[-1]:.2f}")
        crit_time = None
        anim_idt = None
    else:
        crit_time = tt[id_crit]
        anim_idt = id_crit + 3
        print(f"Instability after {id_crit:d} timesteps, t={crit_time:.2f}")
    if ani:
        animate_u(uunt[:anim_idt:4,:], xx, tt[:anim_idt:4], a, exact=u_exact)
    return crit_time, id_crit, tt, uunt, xx


## 1 – Numerical instability: violent development

1. Repeat the numerical simulation carried out __in exercise [ex_2a](https://github.com/AST-Course/AST5110/blob/main/ex_2a.ipynb)__, but now take $a = 1$ (it was $a = −1$); (use a moderate number of intervals, like, e.g., 128). Check out what happens. After how many timesteps does instability become evident?

Repeat the experiment with 1024 points (and afterward with any other power of 2 you may want to use). Is the experiment stable now? After how many timesteps does the instability become evident now?

2. Still for $a = 1$, use for the spatial differentiation backward finite differencing, i.e.:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_i-u_{i-1}}{\Delta x}  \tag{1}$$

Fill in the `nm_lib` function `deriv_upw`, and use the `lambda` function `ddx` and select the proper limits for `bnd_limits`. 

__taking care of also changing the implementation of the boundary condition.__ Does the unstable character of the code change?

### Testing instability with downwind:

In [ ]:
if __name__ == "__main__":
    analyse_instability(128)
    analyse_instability(1024)
    analyse_instability(1024*2, animate=False)
    analyse_instability(1024*4, animate=False)

### Testing instability with upwind:

In [ ]:
if __name__ == "__main__":
    analyse_instability(64, ddx= nm.deriv_upw, bnd_limits=[1,0], animate=False)
    analyse_instability(128, ddx= nm.deriv_upw, bnd_limits=[1,0])

## 2 – Centered differences

Use now for the spatial derivation _centered finite differences_, i.e.:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_{i+1}-u_{i-1}}{2\Delta x}   \tag{2}$$

(and see the note on boundary conditions below). Fill in `nm_lib` the function `deriv_cent`, and like in the previous case, use the `lambda` function `ddx` and select the proper limits for `bnd_limits`. Use `cfl_cut=0.3`. Is any instability apparent in this case? Does the situation change when you change the sign of the constant $a$?

Note: In this case, the periodicity boundary condition can be implemented as follows: define $xx$ so that the endpoint $x = x_0$ of the domain coincides with $xx[1]$ (i.e., the second component of the array) and the endpoint $x = x_f$ coincides with the last element in the array [i.e., $xx[nump-1]$]. For the boundary condition, you can do the following: assume you are calling $uun$ the array at time $t + \Delta t$. Then the boundary condition is imposed by specifying $uun[0] = uun[nump-2]$ and $uun[nump-1] = uun[1]$.

In [ ]:
if __name__ == "__main__":
    nt = 2000
    analyse_instability(128,nt=nt,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=False)
    analyse_instability(128,nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=False)
    analyse_instability(64,nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=False)
    analyse_instability(256,nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=False)
    analyse_instability(256*2,nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=False)

## 3 – The stability of the non-centered finite-differences schemes

In the previous exercises, we saw that when $a > 0$, a _backward-oriented_ finite-difference scheme yields stability. However, a crucial component in the problem was to give a specific value for $\Delta t$, namely  $t = 0.98 x/|a|$. Would it have been wise to choose a larger or smaller $\Delta t$? Let us check that $\Delta t$ cannot be chosen arbitrarily large: run the program, but now writing

1. $\Delta t = 0.5 \frac{\Delta x}{a}$;
2. $\Delta t = 0.99 \frac{\Delta x}{a}$;
3. $\Delta t = 1.01 \frac{\Delta x}{a}$;
4. $\Delta t = 2 \frac{\Delta x}{a}$;

and check if those values maintain the good stability properties of the code. For example, does there seem to be a threshold in $\Delta t$ for the instability? Note that you need to define `cfl_cut` to 0.5, 0.99, 1.01, and 2.0.

In [ ]:
if __name__ == "__main__":
    reload(nm)
    cfl_cuts = [0.5, 0.99, 1.01, 2.0]
    nint = 64
    a = 1
    ddx = nm.deriv_dnw if a<0 else nm.deriv_upw
    bnd_limits = [0,1] if a<0 else [1,0]
    for cfl_cut in cfl_cuts:
        nt = int(52/(cfl_cut*5.2/nint)) # Number of time steps for the propagating wave to travel through the domain 10 times
        analyse_instability(nint,nt,a=a,ddx=ddx,bnd_limits=bnd_limits, cfl_cut=cfl_cut, animate=False)

In [ ]:
if __name__ == "__main__":
    cfl_cut = 1.01
    nint = 64
    a = 1
    ddx = nm.deriv_dnw if a<0 else nm.deriv_upw
    bnd_limits = [0,1] if a<0 else [1,0]
    nt = int(52/(cfl_cut*5.2/nint)) # Number of time steps for the propagating wave to travel through the domain 10 times
    analyse_instability(nint,nt,a=a,ddx=ddx,bnd_limits=bnd_limits, cfl_cut=cfl_cut, animate=True)

## 4- Optional (A). 

Consider now the following Burgers’ equation, i.e.,

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{3}$$

can be seen as yet another case of the equation solved in the previous exercises of this
series by writing:

$$a(x,t,u) = u  \tag{4}$$

The importance of this equation in physics derives in part from the fact that it describes
the motion of a non-accelerated fluid with an arbitrary velocity field at time $t = 0$ and
because it contains the possibility of spontaneous formation of discontinuities. (\*)

We can simply solve equation (1) by modifying the program developed for the previous exercises but now writing $a(x_i, t^n) = u^n_i$. Carry out that modification, and run the program in the domain $x \in (x_0, x_f)$ with $x_0 = −1.4$, $x_f = 2.0$ with the initial condition:

$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right) \right] + B \tag{5}$$

whereby $A = 1.0$, $x_c = 0.70$, $W = 0.1$, $B = 0.3$. Let the solution evolve until time $t_f = 100$. Explain in physical (or mathematical) terms the solution you get. Change to $A = −0.02$ and explain the result. For this exercise, fill in `nm_lib` functions `evolv_uadv_burgers` and `step_uadv_burgers`. 

In [ ]:
def get_uadv_t0(xx: np.ndarray, A: float = 1.0, W: float = 0.1, B: float = 0.3, xc: float = 0.7) -> np.ndarray:
    tanh_term = np.tanh((xx+xc)/W) - np.tanh((xx-xc)/W)
    return A*tanh_term + B

if __name__ == "__main__":
    x0 = -1.4
    xf = 2.0
    nint = 256*2
    tf = 100
    xx, _ = nm.get_xx(nint, x0, xf)
    uut0 = get_uadv_t0(xx)
    cfl_cut = 0.98
    nt = int(round(tf/(cfl_cut*nm.cfl_adv_burger(np.mean(uut0), xx)))) + 1
    # nt = 2000
    *_, tt, uunt, xx = analyse_instability(nint=nint, nt=nt, x0=x0, xf=xf, a=None, crit_value=3,
                        ddx = nm.deriv_upw, bnd_limits=[1,0], cfl_cut=cfl_cut,
                        init_func= lambda x: get_uadv_t0(x), evolv_func=nm.evolv_uadv_burgers, u_exact=None, animate=False)
    # animate the start
    animate_u(uunt[:int(nint/2):8], xx, tt[:int(nint/2):8], a=None, exact=None)
    # animate the end
    animate_u(uunt[-int(nint/2)::8], xx, tt[-int(nint/2)::8], a=None, exact=None, initial=uunt[0,:])

In [ ]:
if __name__ == "__main__":
    x0 = -1.4
    xf = 2.0
    nint = 256
    tf = 100
    A = -0.02
    xx, _ = nm.get_xx(nint, x0, xf)
    uut0 = get_uadv_t0(xx, A=A)
    cfl_cut = 0.98
    tt, uunt = nm.evolv_uadv_burgers(xx, uut0, ddx=nm.deriv_upw,bnd_limits=[1,0], end_time=tf)
    # animate the start
    animate_u(uunt[:int(nint/2):8], xx, tt[:int(nint/2):8], a=None, exact=None)
    # animate the end
    animate_u(uunt[-int(nint/4):], xx, tt[-int(nint/4):], a=None, exact=None, initial=uunt[0,:])

### Comments:

In the numerical solution, we see the initial condition patently evolving into something that looks like a discontinuity. We understand that this discontinuity is formed because the characteristic curves $dx_p/dt = a(x, t, u) = u$ are more inclined (i.e., faster) in a spacetime diagram when starting at elements with larger $u$: since the solution is constant along those curves, the faster elements, therefore, catch those in front of them which have lower $u$ (as is the case in the solution you have found numerically here) – so the solution must steepen when that happens.

Mathematically and physically, we should not be too surprised: we know that, for instance, in gas dynamics, weird nonlinear phenomena take place, some of which (the shocks) have to do with the formation of discontinuities or sharp transitions. Numerically, though, we ought to be rather surprised for various reasons:
    
a. the numerical calculation of a very large value of the derivative (as is surely the case across the big jump forming in our calculation) cannot be very accurate. It might even happen, we think, that the program crashed because of _NaNs_ or exceptions, etc, occurring in the calculation. But, in fact, our program happily calculates away ... seemingly forever.

b. when a discontinuity forms, the exact mathematical solution, strictly speaking, cannot be calculated in a simple way anymore: when there is a discontinuity, one must take into account what is called the jump relations, also called internal boundary conditions, across the discontinuity: they give the mutual relation of the variables on either side of the discontinuity. A solution obtained in this way is called a __weak solution__ in mathematics.

c. we finally ask ourselves what the mathematically-exact weak solution would be in this case and whether our simple numerical scheme provides a solution near the exact one despite the obvious difficulty of handling large jumps across the near-discontinuity.

We have to leave these as open questions for discussion at a later point. 

As a final comment, consider the consequences of the fact that the acceleration in gas dynamics contains a term of the general form $u$ $du/dx$ .... Much of the beautiful physics occurring in the universe is due to non-linearities like this one.

## 4- Optional (B) 

For this exercise, do first [ex_4a](https://github.com/AST-Course/AST5110/blob/main/ex_4a.ipynb). 
Consider the same setup as the previous exercise. But now, solve it using the upwind method: 

$$u^{n+1}_j = u^{n}_j - \frac{\Delta t}{\Delta x} u^{n}_j(u^{n}_j- u^{n}_{j+1})$$

And using the Lax method implemented in [ex_4a](https://github.com/AST-Course/AST5110/blob/main/ex_4a.ipynb). What do you see? Argue why you think the solution is not correct. __Hint__ In the first exercise, we disccused about flux conservation.

### Comments:
    
See [Stagger mesh](https://github.com/AST-Course/AST5110/wiki/Staggered-mesh) documentation on how a staggered mesh allows keeping the conservation. 